In [2]:
333*333/4

27722.25

In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import random as random
import csv
import heapq
from tqdm import tqdm

In [3]:
# Hyper parameters
EPISODES = 50 # times every setting is replicated
#IF_WRITEFILE = True
IF_WRITEFILE = False
RESULT_ADD = '~/'

In [4]:
N = 100  # total number of bikes in the QN
A = 4  # A for areas, indicates the number of areas and the action space


# initialize parameters
R = {
        'lambda0' : 20.0,
        'lambda1' : 20.0,
        'lambda2' : 20.0,
        'lambda3' : 20.0,
        'car_move': 1.0,
        'gamma' : 1.0,
        'c_server' : 1,
        'phi' : 1.0,
        'f_server' : 1,
        'delta' : 1.0,
        'd_server' : 1,
        'broken' : 0,
        'period' : 90, #the maximal time for a carrier to move among areas
       }
#queues: A0, A1, C, F, D, R00, R01, R10, R11
P = [
        [0.1, 0.4, 0.4, 0.1], 
        [0.4, 0.1, 0.1, 0.4], 
        [0.4, 0.1, 0.1, 0.4],
        [0.1, 0.4, 0.4, 0.1],
      ]

In [5]:
list(range(10, 20, 10))

[10]

In [8]:
# theoretical QN
# simulate situation with collect, fix and redistribute
# in this version, bikes found broken after a customer RIDING


# BikeNet is the class for simulate the CQN
# simulate by discrete event, in this program called a step
# the average customer served on time is out research object

class BikeNet():
    
    # initiate the parameters in this function
    def __init__(self, N, A, R, P, warmup_time, run_time):
        '''
        N: number of bikes in the CQN
        A: number of areas in the CQN
        R: rates of the system
        P: matrix of transit probability, including to broken
        warmup_time: the time to warm up the system
        run_time: the period of time during which get the performance
        '''
        
        self.N = N
        self.A = A
        self.R = R
        self.P = P
        self.warmup_time = warmup_time
        self.run_time = run_time
        self.time_limit = warmup_time + run_time
#         self.edge = int(self.A**0.5)
        self.areas = list(range(A))

        self.served_customers = 0

    def reset(self):
        self.T = 0 # time cursor to indicate the system time in this episode, at the end of every episode should be default to zero
        self.served_customers = 0 # the performance indicator
        self.C = [] # time to be empty for collecting queue
        self.F = [] # time to be empty for fixing queue
        self.D = [] # time to be empty for distributing queue
        [heapq.heapify(_) for _ in [self.C, self.F, self.D]]
        # states: devide into 2 parts
        # A0, ... AA, C, F, D: 1-d list
        # [[R00, ..., R(0,A-1)1], ..., [R(A-1,0), ..., R(A-1,A-1)]]: 2-d list for all riding queues
        # self.state = [int(self.N/self.A)]*self.A + [0]*(self.A**2+3) # initiate the state list
        self.state1 = [int(self.N/self.A)]*self.A + [0]*3 # the last 3 numbers are for C, F, and D
        self.state2 = [[0]*self.A for i in range(self.A)]
        self.scheduler = []
        heapq.heapify(self.scheduler)
        for i in range(self.A):
            # state: [time, type, start, terminal]
            # initially generate four events of arriving at each area
            heapq.heappush(self.scheduler, [random.expovariate(self.R['lambda'+str(i)]), -1, i, 0])
        return self.state1, self.state2, self.T

    def simulate(self, writefile):
        if writefile:
            with open(RESULT_ADD, 'w') as fout:
                writer = csv.writer(fout)
                writer.writerow(self.R)
                writer.writerow(self.R.values())
                writer.writerow(self.reset())
                while self.T <= self.time_limit:
                    writer.writerow(self.step())
        else:
            self.reset()
            while self.T <= self.time_limit:
                self.step()
        return self.served_customers / (self.T-self.warmup_time)
                
    def get_riding_time(self, s, t):
        '''
        |0|1|
        |2|3|
        '''
        if s==t: r = 0.5
        elif abs(s-t) == 2: return 2.0
        else: r = 1.0
        return random.expovariate(r)

        
    def add_event(self, kind):
        if kind == 2: 
            next_time = random.expovariate(self.R['gamma'])
            next_time += self.T if self.state1[-3] <= self.R['c_server'] else heapq.heappop(self.C)
            heapq.heappush(self.C, next_time)
            start, end = 'c', 'f'
        elif kind == 3: 
            next_time = random.expovariate(self.R['phi'])
            next_time += self.T if self.state1[-2] <= self.R['f_server'] else heapq.heappop(self.F)
            heapq.heappush(self.F, next_time)
            start, end = 'f', 'd'
        elif kind == 4: 
            next_time = random.expovariate(self.R['delta'])
            next_time += self.T if self.state1[-1] <= self.R['d_server'] else heapq.heappop(self.D)
            heapq.heappush(self.D, next_time)
            start, end = 'd', random.choice(list(range(self.A)))
        heapq.heappush(self.scheduler, [next_time, kind, start, end])
        
    
    def step(self):

        event = self.scheduler[0]
        self.T, kind, start, terminal = event[0], event[1], event[2], event[3]

        '''
        kind of events:
        -1: customer ride a bike away
         1: a bike arrives at any area
         2: a bike finishes at collecting queue
         3: a bike finishes at fixed queue
         4: a bike finishes at distributed queue
        '''
        if kind == 1: # 顾客骑行到达
            self.state2[start][terminal] -= 1
            if random.random()<self.R['broken']:
                self.state1[-3] += 1
                heapq.heappop(self.scheduler)
                self.add_event(2)
            else:
                self.state1[terminal] += 1
                heapq.heappop(self.scheduler)
        elif kind == 2:
            self.state1[-3] -= 1
            self.state1[-2] += 1
            heapq.heappop(self.scheduler)
            if self.state1[-3] < self.R['c_server']: heapq.heappop(self.C)
            self.add_event(3)
        elif kind == 3:
            self.state1[-2] -= 1
            self.state1[-1] += 1
            heapq.heappop(self.scheduler)
            if self.state1[-2] < self.R['f_server']: heapq.heappop(self.F)
            self.add_event(4)
        elif kind == 4:
            self.state1[-1] -= 1
            self.state1[terminal] += 1
            heapq.heappop(self.scheduler)
            if self.state1[-1] < self.R['d_server']: heapq.heappop(self.D)
        else:# 顾客到达
            if self.state1[start] == 0:  # 但没车
                heapq.heappop(self.scheduler)
                next_time = random.expovariate(self.R['lambda'+str(start)]) + self.T
                heapq.heappush(self.scheduler, [next_time, -1, start, 0])
            else:
                if self.T>self.warmup_time: 
                    self.served_customers += 1
                    #result.append(self.served_customers / (self.T-self.warmup_time))
                # if self.T>self.warmup_time: result.append(self.T)
                
                self.state1[start] -= 1
                target = random.choices(self.areas, weights=self.P[start], k=1)[0]
                self.state2[start][target] += 1
                
                heapq.heappop(self.scheduler)
                next_time = self.get_riding_time(start, target) + self.T
                heapq.heappush(self.scheduler, [next_time, 1, start, target])
                next_time = random.expovariate(self.R['lambda'+str(start)]) + self.T
                heapq.heappush(self.scheduler, [next_time, -1, start, 0])


        return self.state1, self.state2, self.T



warmup_time = 200
run_time = 600

env = BikeNet(N=N,
              A=A,
              R=R,
              P=P,
              warmup_time=warmup_time,
              run_time=run_time)

#random.seed(1)

result = []

%time env.simulate(IF_WRITEFILE)

# for r in tqdm(range(1000, 1001)):
#     for i in range(1000, 1001):
#         env.R['lambda0']=env.R['lambda1']=env.R['lambda2']=env.R['lambda3'] = r
#         env.N = i
#         result.append([r, i, env.simulate(IF_WRITEFILE)])


# test the influence of certain parameter
# for i in tqdm(range(1,100)):
#     env.N = i
#     result.append(env.simulate(IF_WRITEFILE))
# plt.plot(result)
# plt.show()

CPU times: user 437 ms, sys: 4.14 ms, total: 441 ms
Wall time: 472 ms


59.45560438269296

In [331]:
result

[[1000, 1000, 665.790799689757]]

In [35]:
# real
# the influence of phi
# in this version, carrier moves among areas to collect broken bikes
# then brings them to the repair center, and bring the normal bikes back to areas
class BikeNet():
    def __init__(self, N, A, R, P, warmup_time, run_time):
        self.N = N
        self.A = A
        self.R = R
        self.P = P
        self.warmup_time = warmup_time
        self.run_time = run_time
        self.time_limit = warmup_time + run_time
        #self.C = 0
        self.F = 0
        #self.D = 0
        self.capacity = 12
        self.areas = list(range(A))

        self.serverd_customers = 0

    def reset(self):
        self.T = 0
#         self.C = 0
        self.F = 0
#         self.D = 0
        self.serverd_customers = 0
        #queues: A0, A1, B0, B1, FN, FB, R00, R01, R10, R11
        self.state = [int(self.N/self.A)]*self.A + [0]*(self.A**2+2+self.A)
        self.scheduler = []
        heapq.heapify(self.scheduler)
        for i in range(self.A):
            # state: [time, type, start, terminal]
            heapq.heappush(self.scheduler, [random.expovariate(self.R['lambda'+str(i)]), -1, i, 0])
        #event of the carrier, [time, kind, place, [normal, broken, time]]
        heapq.heappush(self.scheduler, [0, 0, 0, [0,0,self.T+self.R['period']]])
        return self.state + [self.T]

    def simulate(self):
        with open('C:/Rebalancing/data/simulationResult/central/data/central_broken_real.csv', 'w') as fout:
            writer = csv.writer(fout)
            writer.writerow(self.R)
            writer.writerow(self.R.values())
            writer.writerow(self.reset())
            while self.T <= self.time_limit:
                writer.writerow(self.step())
        return self.serverd_customers / (self.T-10000)
                
    def get_rho(self, path):
        s, t = int(path[0]), int(path[1])
        if s==t: return 0.5
        elif abs(s-t) == 2: return 2.0
        else: return 1.0
                
    def get_index(self, target):
        if target == 'c': return 4
        elif target == 'f': return 5
        elif target == 'd': return 6
        else:
            s, t = int(target[0]), int(target[1])
            return 2*self.A + 2 + 4*s + t
        
    def add_event(self, kind, s):
        if kind == 2:
            next_time = random.expovariate(self.R['gamma']) + self.T
            start, end = s, 'f'
        elif kind == 3:
            next_time = random.expovariate(self.R['phi']) + max(self.T, self.F)
            self.F, start, end = next_time, 'f', 'd'
        elif kind == 0:
            next_time = random.expovariate(self.R['delta']) + self.T
            start, end = random.choice(list(range(self.A))), [s, 0, self.T+self.R['period']]
        heapq.heappush(self.scheduler, [next_time, kind, start, end])
        
    
    def step(self):

        event = self.scheduler[0]
        self.T, kind, start, terminal = event[0], event[1], event[2], event[3]

        '''kind:
        -1: customer ride a bike away
         0: carrier arrives as a area
         1: a bike arrives
         2: carrier full of broken bikes arrives as the repairing center
         3: a bike was fixed
        '''
        if kind == 0: #carrier 正在areas之间逡巡中，鱼戏莲叶南，鱼戏莲叶北
            normal, broken, end_time = terminal[0], terminal[1], terminal[2]
            if normal > 0: #如果有好车，先把好车放下
                #print(self.scheduler[0])
                #print('holla', normal, broken)
                number = min(normal, int(self.capacity/self.A))
                self.state[start] += number
                self.scheduler[0][3][0] -= number
                normal -= number
                #print(self.scheduler[0])
                #print(self.state, '\n')
            if self.state[self.A+start]>0 and broken < self.capacity: #如果有坏车，并且运载车没满，把坏车装上运载车
                number = min(self.capacity - broken, self.state[self.A + start])
                self.state[self.A+start] -= number
                self.scheduler[0][3][1] += number
                broken += number
            if broken == self.capacity or (self.T>end_time and normal==0):
                heapq.heappop(self.scheduler)
                self.add_event(2, broken)
            else:
                self.scheduler[0][0] += random.expovariate(self.R['car_move'])
                self.scheduler[0][2] = (start+1)%4
                heapq.heapify(self.scheduler) 
        elif kind == 1: # 顾客骑行到达
            self.state[self.get_index(start)] -= 1
            if random.random()<self.R['broken']:
                self.state[self.A+terminal] += 1
                heapq.heappop(self.scheduler)
            else:
                self.state[terminal] += 1
                heapq.heappop(self.scheduler)
        elif kind == 2:
            if self.state[self.A * 2 + 1]==0:
                self.add_event(3, start)
            self.state[self.A * 2 + 1] += start
            heapq.heappop(self.scheduler)
            number = min(self.capacity, self.state[self.A * 2])
            self.add_event(0, number)
            self.state[self.A * 2] -= number
        elif kind == 3:
            #print(self.state)
            self.state[self.A * 2] += 1
            self.state[self.A * 2 + 1] -= 1
            heapq.heappop(self.scheduler)
            if self.state[self.A * 2 + 1] > 0:
                self.add_event(3, start)
                #print('next fix')
            #print(self.state)
        else:# 顾客到达
            if self.state[start] == 0:  # 但没车
                heapq.heappop(self.scheduler)
                next_time = random.expovariate(self.R['lambda'+str(start)]) + self.T
                heapq.heappush(self.scheduler, [next_time, -1, start, 0])
            else:
                target = str(start)+ str(np.random.choice(self.areas, 1, p=self.P[start])[0])
                if self.T>10000: self.serverd_customers += 1
                self.state[start] -= 1
                self.state[self.get_index(target)] += 1
                heapq.heappop(self.scheduler)
                next_time = random.expovariate(self.get_rho(target)) + self.T
                heapq.heappush(self.scheduler, [next_time, 1, target, int(target[1])])
                next_time = random.expovariate(self.R['lambda'+str(start)]) + self.T
                heapq.heappush(self.scheduler, [next_time, -1, start, 0])


        return self.state+[self.T]

random.seed(1)
N = 100  # total number of bikes in the QN
A = 4  # A for areas, indicates the number of areas and the action space
R = rate
P = pij


run_time = 20000

env = BikeNet(N=N,
              A=A,
              R=R,
              P=P,
              #repair=t_repair,
              warmup_time=warmup_time,
              run_time=run_time)
              #start_position=0)

result = []
#for j in tqdm(range(1,20)):
#    env.R['gamma'] = 0.5 + j*0.5
for i in range(21):
    env.R['broken'] = i*0.05
    result.append(env.simulate())
    
plt.plot(result)
plt.show()

13.215946851452115


In [5]:
# integrated server ruuning version
# with multiprocessing
# central no broken same lambda


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import random as random
import csv
import heapq
from tqdm import tqdm

from multiprocessing import Pool

# Hyper parameters
EPISODES = 50 # times every setting is replicated
IF_WRITEFILE = True
#IF_WRITEFILE = False
RESULT_ADD = '/Users/valarian/SJTU/SJTU/毕业论文/Data/result/data_仿真结果/' + 'central_no_broken_same_lambda.csv'

WARMUP_TIME = 200
RUN_TIME = 500

N = 100  # total number of bikes in the QN
A = 4  # A for areas, indicates the number of areas and the action space


# initialize parameters
R = {
        'lambda0' : 20.0,
        'lambda1' : 20.0,
        'lambda2' : 20.0,
        'lambda3' : 20.0,
        'car_move': 1.0,
        'gamma' : 1.0,
        'c_server' : 1,
        'phi' : 1.0,
        'f_server' : 1,
        'delta' : 1.0,
        'd_server' : 1,
        'broken' : 0,
        'period' : 90, #the maximal time for a carrier to move among areas
       }
#queues: A0, A1, C, F, D, R00, R01, R10, R11
P = [
        [0.1, 0.4, 0.4, 0.1], 
        [0.4, 0.1, 0.1, 0.4], 
        [0.4, 0.1, 0.1, 0.4],
        [0.1, 0.4, 0.4, 0.1],
      ]

# theoretical QN
# simulate situation with collect, fix and redistribute
# in this version, bikes found broken after a customer RIDING


# BikeNet is the class for simulate the CQN
# simulate by discrete event, in this program called a step
# the average customer served on time is out research object

class BikeNet():
    
    # initiate the parameters in this function
    def __init__(self, N, A, R, P, warmup_time, run_time):
        '''
        N: number of bikes in the CQN
        A: number of areas in the CQN
        R: rates of the system
        P: matrix of transit probability, including to broken
        warmup_time: the time to warm up the system
        run_time: the period of time during which get the performance
        '''
        
        self.N = N
        self.A = A
        self.R = R
        self.P = P
        self.warmup_time = warmup_time
        self.run_time = run_time
        self.time_limit = warmup_time + run_time
#         self.edge = int(self.A**0.5)
        self.areas = list(range(A))

        self.served_customers = 0

    def reset(self):
        self.T = 0 # time cursor to indicate the system time in this episode, at the end of every episode should be default to zero
        self.served_customers = 0 # the performance indicator
        self.C = [] # time to be empty for collecting queue
        self.F = [] # time to be empty for fixing queue
        self.D = [] # time to be empty for distributing queue
        [heapq.heapify(_) for _ in [self.C, self.F, self.D]]
        # states: devide into 2 parts
        # A0, ... AA, C, F, D: 1-d list
        # [[R00, ..., R(0,A-1)1], ..., [R(A-1,0), ..., R(A-1,A-1)]]: 2-d list for all riding queues
        # self.state = [int(self.N/self.A)]*self.A + [0]*(self.A**2+3) # initiate the state list
        self.state1 = [int(self.N/self.A)]*self.A + [0]*3 # the last 3 numbers are for C, F, and D
        self.state2 = [[0]*self.A for i in range(self.A)]
        self.scheduler = []
        heapq.heapify(self.scheduler)
        for i in range(self.A):
            # state: [time, type, start, terminal]
            # initially generate four events of arriving at each area
            heapq.heappush(self.scheduler, [random.expovariate(self.R['lambda'+str(i)]), -1, i, 0])
        return self.state1, self.state2, self.T

    def simulate(self, writefile):
        if writefile:
            with open(RESULT_ADD, 'a') as fout:
                writer = csv.writer(fout)
                #writer.writerow(self.R)
                #writer.writerow(self.R.values())
                #writer.writerow(self.reset())
                self.reset()
                while self.T <= self.time_limit:
                    self.step()
                writer.writerow([self.R['lambda0'], self.N, self.served_customers / (self.T-self.warmup_time)])
        else:
            self.reset()
            while self.T <= self.time_limit:
                self.step()
            return self.served_customers / (self.T-self.warmup_time)
                
    def get_riding_time(self, s, t):
        '''
        |0|1|
        |2|3|
        '''
        if s==t: r = 0.5
        elif abs(s-t) == 2: return 2.0
        else: r = 1.0
        return random.expovariate(r)

        
    def add_event(self, kind):
        if kind == 2: 
            next_time = random.expovariate(self.R['gamma'])
            next_time += self.T if self.state1[-3] <= self.R['c_server'] else heapq.heappop(self.C)
            heapq.heappush(self.C, next_time)
            start, end = 'c', 'f'
        elif kind == 3: 
            next_time = random.expovariate(self.R['phi'])
            next_time += self.T if self.state1[-2] <= self.R['f_server'] else heapq.heappop(self.F)
            heapq.heappush(self.F, next_time)
            start, end = 'f', 'd'
        elif kind == 4: 
            next_time = random.expovariate(self.R['delta'])
            next_time += self.T if self.state1[-1] <= self.R['d_server'] else heapq.heappop(self.D)
            heapq.heappush(self.D, next_time)
            start, end = 'd', random.choice(list(range(self.A)))
        heapq.heappush(self.scheduler, [next_time, kind, start, end])
        
    
    def step(self):

        event = self.scheduler[0]
        self.T, kind, start, terminal = event[0], event[1], event[2], event[3]

        '''
        kind of events:
        -1: customer ride a bike away
         1: a bike arrives at any area
         2: a bike finishes at collecting queue
         3: a bike finishes at fixed queue
         4: a bike finishes at distributed queue
        '''
        if kind == 1: # 顾客骑行到达
            self.state2[start][terminal] -= 1
            if random.random()<self.R['broken']:
                self.state1[-3] += 1
                heapq.heappop(self.scheduler)
                self.add_event(2)
            else:
                self.state1[terminal] += 1
                heapq.heappop(self.scheduler)
        elif kind == 2:
            self.state1[-3] -= 1
            self.state1[-2] += 1
            heapq.heappop(self.scheduler)
            if self.state1[-3] < self.R['c_server']: heapq.heappop(self.C)
            self.add_event(3)
        elif kind == 3:
            self.state1[-2] -= 1
            self.state1[-1] += 1
            heapq.heappop(self.scheduler)
            if self.state1[-2] < self.R['f_server']: heapq.heappop(self.F)
            self.add_event(4)
        elif kind == 4:
            self.state1[-1] -= 1
            self.state1[terminal] += 1
            heapq.heappop(self.scheduler)
            if self.state1[-1] < self.R['d_server']: heapq.heappop(self.D)
        else:# 顾客到达
            if self.state1[start] == 0:  # 但没车
                heapq.heappop(self.scheduler)
                next_time = random.expovariate(self.R['lambda'+str(start)]) + self.T
                heapq.heappush(self.scheduler, [next_time, -1, start, 0])
            else:
                if self.T>self.warmup_time: 
                    self.served_customers += 1
                    #result.append(self.served_customers / (self.T-self.warmup_time))
                # if self.T>self.warmup_time: result.append(self.T)
                
                self.state1[start] -= 1
                target = random.choices(self.areas, weights=self.P[start], k=1)[0]
                self.state2[start][target] += 1
                
                heapq.heappop(self.scheduler)
                next_time = self.get_riding_time(start, target) + self.T
                heapq.heappush(self.scheduler, [next_time, 1, start, target])
                next_time = random.expovariate(self.R['lambda'+str(start)]) + self.T
                heapq.heappush(self.scheduler, [next_time, -1, start, 0])


        #return self.state1, self.state2, self.T

def run(i):
    env = BikeNet(N=N,
                  A=A,
                  R=R,
                  P=P,
                  warmup_time=WARMUP_TIME,
                  run_time=RUN_TIME)

    env.simulate(IF_WRITEFILE)
    #print('finied %d'%(i))

import time
    
if __name__ == '__main__':
    run(1)
    run(2)
#         p = Pool(7)
#         for i in range(7):
#             p.apply_async(run, args=(i,))
#         p.close()
#         p.join()
    

In [12]:
import scipy as sp

In [35]:
for i in range(5):
    a, b, c, d = sp.stats.truncnorm.rvs(-130, 0, loc=400, scale=3, size=4)
    print(a,b,c,d)

394.82269122180577 393.6977736226013 396.4665781535696 396.89382825640706
397.3993677562349 399.6715126127353 397.4991108902301 399.274992107689
398.75682263127595 399.8587495426415 397.5115562304622 396.52135093506143
397.9511036454478 396.29547850286826 390.4809649114351 398.34417173888016
397.6347326626844 398.10553411657776 398.42661095040296 398.4542207886865


In [42]:
# integrated server ruuning version
# with multiprocessing
# central no broken vary lambda

# Solution
# We set the N to be 100
# summation of increases from 1 to 400
# draw 30 sample from a trancated normal distribution as [0, +∞]
# with mean of summation, standard variance in [0, 20, 40, 60]
# variance equals to 0 is the same as the previous analysis


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import random as random
import csv
import heapq
from tqdm import tqdm

from multiprocessing import Pool

# Hyper parameters
EPISODES = 50 # times every setting is replicated
IF_WRITEFILE = True
#IF_WRITEFILE = False
RESULT_ADD = '/Users/valarian/SJTU/SJTU/毕业论文/Data/result/data_仿真结果/central/' + 'central_no_broken_vary_lambda.csv'

WARMUP_TIME = 2000
RUN_TIME = 5000

N = 100  # total number of bikes in the QN
A = 4  # A for areas, indicates the number of areas and the action space


# initialize parameters
R = {
        'lambda0' : 20.0,
        'lambda1' : 20.0,
        'lambda2' : 20.0,
        'lambda3' : 20.0,
        'car_move': 1.0,
        'gamma' : 1.0,
        'c_server' : 1,
        'phi' : 1.0,
        'f_server' : 1,
        'delta' : 1.0,
        'd_server' : 1,
        'broken' : 0,
        'sv' : 0, # standard variance
        'period' : 90, #the maximal time for a carrier to move among areas
       }
#queues: A0, A1, C, F, D, R00, R01, R10, R11
P = [
        [0.1, 0.4, 0.4, 0.1], 
        [0.4, 0.1, 0.1, 0.4], 
        [0.4, 0.1, 0.1, 0.4],
        [0.1, 0.4, 0.4, 0.1],
      ]

# theoretical QN
# simulate situation with collect, fix and redistribute
# in this version, bikes found broken after a customer RIDING


# BikeNet is the class for simulate the CQN
# simulate by discrete event, in this program called a step
# the average customer served on time is out research object

class BikeNet():
    
    # initiate the parameters in this function
    def __init__(self, N, A, R, P, warmup_time, run_time):
        '''
        N: number of bikes in the CQN
        A: number of areas in the CQN
        R: rates of the system
        P: matrix of transit probability, including to broken
        warmup_time: the time to warm up the system
        run_time: the period of time during which get the performance
        '''
        
        self.N = N
        self.A = A
        self.R = R
        self.P = P
        self.warmup_time = warmup_time
        self.run_time = run_time
        self.time_limit = warmup_time + run_time
#         self.edge = int(self.A**0.5)
        self.areas = list(range(A))

        self.served_customers = 0

    def reset(self):
        self.T = 0 # time cursor to indicate the system time in this episode, at the end of every episode should be default to zero
        self.served_customers = 0 # the performance indicator
        self.C = [] # time to be empty for collecting queue
        self.F = [] # time to be empty for fixing queue
        self.D = [] # time to be empty for distributing queue
        [heapq.heapify(_) for _ in [self.C, self.F, self.D]]
        # states: devide into 2 parts
        # A0, ... AA, C, F, D: 1-d list
        # [[R00, ..., R(0,A-1)1], ..., [R(A-1,0), ..., R(A-1,A-1)]]: 2-d list for all riding queues
        # self.state = [int(self.N/self.A)]*self.A + [0]*(self.A**2+3) # initiate the state list
        self.state1 = [int(self.N/self.A)]*self.A + [0]*3 # the last 3 numbers are for C, F, and D
        self.state2 = [[0]*self.A for i in range(self.A)]
        self.scheduler = []
        heapq.heapify(self.scheduler)
        for i in range(self.A):
            # state: [time, type, start, terminal]
            # initially generate four events of arriving at each area
            heapq.heappush(self.scheduler, [random.expovariate(self.R['lambda'+str(i)]), -1, i, 0])
        return self.state1, self.state2, self.T

    def simulate(self, writefile):
        if writefile:
            with open(RESULT_ADD, 'a') as fout:
                writer = csv.writer(fout)
                #writer.writerow(self.R)
                #writer.writerow(self.R.values())
                #writer.writerow(self.reset())
                self.reset()
                while self.T <= self.time_limit:
                    self.step()
                sum_lambda = self.R['lambda0']+self.R['lambda1']+self.R['lambda2']+self.R['lambda3']
                writer.writerow([sum_lambda, self.R['sv'], self.served_customers / (self.T-self.warmup_time)])
        else:
            self.reset()
            while self.T <= self.time_limit:
                self.step()
            return self.served_customers / (self.T-self.warmup_time)
                
    def get_riding_time(self, s, t):
        '''
        |0|1|
        |2|3|
        '''
        if s==t: r = 0.5
        elif abs(s-t) == 2: return 2.0
        else: r = 1.0
        return random.expovariate(r)

        
    def add_event(self, kind):
        if kind == 2: 
            next_time = random.expovariate(self.R['gamma'])
            next_time += self.T if self.state1[-3] <= self.R['c_server'] else heapq.heappop(self.C)
            heapq.heappush(self.C, next_time)
            start, end = 'c', 'f'
        elif kind == 3: 
            next_time = random.expovariate(self.R['phi'])
            next_time += self.T if self.state1[-2] <= self.R['f_server'] else heapq.heappop(self.F)
            heapq.heappush(self.F, next_time)
            start, end = 'f', 'd'
        elif kind == 4: 
            next_time = random.expovariate(self.R['delta'])
            next_time += self.T if self.state1[-1] <= self.R['d_server'] else heapq.heappop(self.D)
            heapq.heappush(self.D, next_time)
            start, end = 'd', random.choice(list(range(self.A)))
        heapq.heappush(self.scheduler, [next_time, kind, start, end])
        
    
    def step(self):

        event = self.scheduler[0]
        self.T, kind, start, terminal = event[0], event[1], event[2], event[3]

        '''
        kind of events:
        -1: customer ride a bike away
         1: a bike arrives at any area
         2: a bike finishes at collecting queue
         3: a bike finishes at fixed queue
         4: a bike finishes at distributed queue
        '''
        if kind == 1: # 顾客骑行到达
            self.state2[start][terminal] -= 1
            if random.random()<self.R['broken']:
                self.state1[-3] += 1
                heapq.heappop(self.scheduler)
                self.add_event(2)
            else:
                self.state1[terminal] += 1
                heapq.heappop(self.scheduler)
        elif kind == 2:
            self.state1[-3] -= 1
            self.state1[-2] += 1
            heapq.heappop(self.scheduler)
            if self.state1[-3] < self.R['c_server']: heapq.heappop(self.C)
            self.add_event(3)
        elif kind == 3:
            self.state1[-2] -= 1
            self.state1[-1] += 1
            heapq.heappop(self.scheduler)
            if self.state1[-2] < self.R['f_server']: heapq.heappop(self.F)
            self.add_event(4)
        elif kind == 4:
            self.state1[-1] -= 1
            self.state1[terminal] += 1
            heapq.heappop(self.scheduler)
            if self.state1[-1] < self.R['d_server']: heapq.heappop(self.D)
        else:# 顾客到达
            if self.state1[start] == 0:  # 但没车
                heapq.heappop(self.scheduler)
                next_time = random.expovariate(self.R['lambda'+str(start)]) + self.T
                heapq.heappush(self.scheduler, [next_time, -1, start, 0])
            else:
                if self.T>self.warmup_time: 
                    self.served_customers += 1
                    #result.append(self.served_customers / (self.T-self.warmup_time))
                # if self.T>self.warmup_time: result.append(self.T)
                
                self.state1[start] -= 1
                target = random.choices(self.areas, weights=self.P[start], k=1)[0]
                self.state2[start][target] += 1
                
                heapq.heappop(self.scheduler)
                next_time = self.get_riding_time(start, target) + self.T
                heapq.heappush(self.scheduler, [next_time, 1, start, target])
                next_time = random.expovariate(self.R['lambda'+str(start)]) + self.T
                heapq.heappush(self.scheduler, [next_time, -1, start, 0])


        #return self.state1, self.state2, self.T

def run():
    env = BikeNet(N=N,
                  A=A,
                  R=R,
                  P=P,
                  warmup_time=WARMUP_TIME,
                  run_time=RUN_TIME)

    for s in range(10, 12):
        my_a, my_b = 0, 3*s
        for sv in [1, 2, 3]: # 1 sigma, 2 sigma, 3 sigma
            mean, stdv = s/4, s/3*sv
            env.R['sv'] = sv
            env.R['lambda0'],env.R['lambda1'],env.R['lambda2'],env.R['lambda3'] = \
                sp.stats.truncnorm.rvs((my_a-mean)/stdv, (my_b-mean)/stdv, loc=mean, scale=stdv, size=4)
            env.simulate(IF_WRITEFILE)
    #print('finied %d'%(i))

import time
    
if __name__ == '__main__':

    
    run()
    

2.1796553329844226
2.1796553329844226
13.921548938821815
13.921548938821815
4.171894970505974
4.171894970505974
1.2958975773387196
1.2958975773387196
3.637567773103388
3.637567773103388
11.466018887288566
11.466018887288566
